# Garbage Collection

In [1]:
import torch as th

In [2]:
import syft as sy

In [3]:
hook = sy.TorchHook(th)

In [37]:
bob = sy.VirtualWorker(hook,id="bob")

In [38]:
x = th.tensor([1,2,3,4]).send(bob)

In [39]:
bob._objects

{34159474489: tensor([1, 2, 3, 4])}

In [40]:
del x

In [41]:
bob._objects

{}

In [42]:
for i in range(1000):
    x = th.tensor([1,2,3,4]).send(bob)

In [43]:
bob._objects

{60849088953: tensor([1, 2, 3, 4])}

In [44]:
del x

In [45]:
bob._objects

{}

# Toy Federated Learning

In [1]:
import torch as th

In [2]:
from torch import nn, optim

In [3]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)

In [4]:
model = nn.Sequential(nn.Linear(2,1),
                      nn.ReLU())

In [5]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [6]:
def training(iterations):
    for iter in range(iterations):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
training(200)

tensor(2.0062)
tensor(1.3645)
tensor(0.9377)
tensor(0.6495)
tensor(0.4535)
tensor(0.3191)
tensor(0.2264)
tensor(0.1618)
tensor(0.1165)
tensor(0.0844)
tensor(0.0616)
tensor(0.0452)
tensor(0.0333)
tensor(0.0247)
tensor(0.0184)
tensor(0.0138)
tensor(0.0103)
tensor(0.0078)
tensor(0.0058)
tensor(0.0044)
tensor(0.0033)
tensor(0.0025)
tensor(0.0019)
tensor(0.0015)
tensor(0.0011)
tensor(0.0008)
tensor(0.0006)
tensor(0.0005)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(9.7364e-05)
tensor(7.4385e-05)
tensor(5.6838e-05)
tensor(4.3436e-05)
tensor(3.3197e-05)
tensor(2.5374e-05)
tensor(1.9396e-05)
tensor(1.4827e-05)
tensor(1.1335e-05)
tensor(8.6654e-06)
tensor(6.6249e-06)
tensor(5.0652e-06)
tensor(3.8727e-06)
tensor(2.9610e-06)
tensor(2.2640e-06)
tensor(1.7311e-06)
tensor(1.3237e-06)
tensor(1.0122e-06)
tensor(7.7391e-07)
tensor(5.9179e-07)
tensor(4.5250e-07)
tensor(3.4599e-07)
tensor(2.6459e-07)
tensor(2.0231e-07)
tensor(1.5468e-07)
tensor(1.1829e-07)
tensor(9.04

In [7]:
data[0]

tensor([1., 1.], grad_fn=<SelectBackward>)

In [12]:
model(data[3])

tensor([2.1861e-08], grad_fn=<ReluBackward0>)

In [51]:
naufil = sy.VirtualWorker(hook, id="naufil")
faizan = sy.VirtualWorker(hook, id="faizan")

In [52]:
data_naufil = data[0:2].send(naufil)
target_naufil = target[0:2].send(naufil)

In [53]:
data_faizan = data[2:4].send(faizan)
target_faizan = target[2:4].send(faizan)

In [54]:
datasets = [(data_naufil,target_naufil),(data_faizan,target_faizan)]

In [55]:
model = nn.Linear(2,1)

In [56]:
opt = optim.SGD(params = model.parameters(), lr = 0.1)

In [57]:
data1, target1 = datasets[0]

In [58]:
data1.location

<VirtualWorker id:naufil #tensors:6>

In [61]:
target1.location

<VirtualWorker id:naufil #tensors:7>

In [62]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0510, -0.0615]], requires_grad=True), Parameter containing:
 tensor([-0.0658], requires_grad=True)]

In [65]:
with th.no_grad():
   model.weight.set_(((alice_model.weight.data + bob_model.weight.data) / 2).get())

NameError: name 'alice_model' is not defined

# Advanced Remote Execution

In [64]:
naufil.clear_objects
faizan.clear_objects

<bound method BaseWorker.clear_objects of <VirtualWorker id:faizan #tensors:2>>

In [53]:
x = th.tensor([1,2,3,4]).send(naufil).send(faizan)

In [54]:
x

(Wrapper)>[PointerTensor | me:92732758512 -> faizan:75315879446]

In [55]:
faizan._objects

{23613880314: (Wrapper)>[PointerTensor | faizan:23613880314 -> naufil:83904550070],
 47735554887: tensor([[1, 0],
         [0, 0]]),
 76176561640: tensor([[1, 0],
         [0, 0]]),
 75315879446: (Wrapper)>[PointerTensor | faizan:75315879446 -> naufil:67688684810]}

In [56]:
naufil._objects

{83904550070: tensor([1, 2, 3, 4]), 49095176888: tensor([[1, 1],
         [0, 1]]), 36408048950: tensor([[1, 1],
         [0, 1]]), 67688684810: tensor([1, 2, 3, 4])}

In [57]:
del x

In [58]:
naufil._objects

{83904550070: tensor([1, 2, 3, 4]), 49095176888: tensor([[1, 1],
         [0, 1]]), 36408048950: tensor([[1, 1],
         [0, 1]]), 67688684810: tensor([1, 2, 3, 4])}

In [59]:
faizan._objects

{23613880314: (Wrapper)>[PointerTensor | faizan:23613880314 -> naufil:83904550070],
 47735554887: tensor([[1, 0],
         [0, 0]]),
 76176561640: tensor([[1, 0],
         [0, 0]]),
 75315879446: (Wrapper)>[PointerTensor | faizan:75315879446 -> naufil:67688684810]}

In [21]:
x

NameError: name 'x' is not defined